In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import imutils

In [2]:
img_yes = '../data/yes/'
img_no = '../data/no/'

yes_files = os.listdir(img_yes)
no_files = os.listdir(img_no)

yes_images, no_images = [], []
print('[!] Reading files with positive diagnostic...')
for file in yes_files:
    yes_images.append(plt.imread(f'{img_yes}{file}'))
print('[!] Positive files read!')

print('[!] Reading files with negative diagnostic...')
for file in no_files:
    no_images.append(plt.imread(f'{img_no}{file}'))
print('[!] Negative files read')

[!] Reading files with positive diagnostic...
[!] Positive files read!
[!] Reading files with negative diagnostic...
[!] Negative files read


In [3]:
def imgs_to_grayscale(img_list):
    for i, img in enumerate(img_list):
        if len(img.shape) == 3:
            img_list[i] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

imgs_to_grayscale(yes_images)
imgs_to_grayscale(no_images)

In [4]:
def threshold_imgs(img_list):
    for i, img in enumerate(img_list):
        tmp = cv2.threshold(img, 45, 255, cv2.THRESH_BINARY)[1]
        tmp = cv2.erode(tmp, None, iterations=2)
        tmp = cv2.dilate(tmp, None, iterations=2)
        img_list[i] = tmp

threshold_imgs(yes_images)
threshold_imgs(no_images)

In [5]:
def find_contours(img_list):
    ret = []
    for i, img in enumerate(img_list):
        cont = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cont = imutils.grab_contours(cont)
        ret.append(max(cont, key=cv2.contourArea))
    return ret

yes_img_cont = find_contours(yes_images)
print(yes_img_cont)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\contours.cpp:197: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'
